In [ ]:
#Main libraries
import gymnasium as gym
from gymnasium import spaces
from gymnasium.utils import seeding
import cosysairsim as airsim
from cosysairsim import MultirotorClient
import numpy as np
import os
from tqdm import tqdm
import csv
import datetime
from torch.utils.tensorboard import SummaryWriter
#General
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
import logging
from logging import handlers
from copy import deepcopy
from itertools import count
import tqdm
import pickle
from pathlib import Path

#PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import MultivariateNormal
import torch.nn.utils as torch_utils

import multiprocessing
multiprocessing.set_start_method("spawn", force=True)

client = airsim.MultirotorClient()
client.confirmConnection()
print("Connected to AirSim!")

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(device)

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)

Connected to AirSim!
cuda


In [ ]:
client = MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
action=[0,0,1,0]
max_angles = np.radians([30, 30, 45])  # Reduced pitch/roll angles
pitch = np.clip(action[0] * max_angles[0], -max_angles[0], max_angles[0])
roll = np.clip(action[1] * max_angles[1], -max_angles[1], max_angles[1])

# Use exponential mapping for throttle to give more control in hover region
throttle = (action[2])+1 / 2  # Centered around 0.5
yaw_rate = np.clip(action[3] * max_angles[2], -max_angles[2], max_angles[2])

# Add small time delay for stability
client.moveByRollPitchYawrateThrottleAsync(
    roll=roll,
    pitch=pitch,
    yaw_rate=yaw_rate,
    throttle=throttle,
    duration=1000  # Reduced duration for more frequent updates
).join()



In [3]:
#Train to keep in air

class CosysAirSimEnv_Basic(gym.Env):
    def __init__(self, training_stage=0):
        super(CosysAirSimEnv_Basic, self).__init__()

        self.client = MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)

        self.action_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-5.0, high=5.0, shape=(18,), dtype=np.float32)

        self.max_steps = 200
        self.current_step = 0
        self.episode = 0
        self.wind_factor = 0.0
        self.last_throttle = 0.6
        self.success_count = 0
        
        self.success_threshold = 10
        
        self.training_stage = training_stage
        self.setup_stage_params()
        self.stage_progress = 0
        self.stage_threshold = 25
        
        self.hover_survival_count = 0
        self.consecutive_success = 0
        self.current_target = self._generate_new_target()
        
        self.all_time_min_distance = float('inf')
        high = np.array([
            1.0, 1.0, 1.0,      # normalized position
            1.0, 1.0, 1.0,      # normalized velocity
            1.0, 1.0, 1.0,      # normalized angular velocity
            3.0, 3.0, 3.0,      # normalized acceleration (in G's)
            1.0, 1.0, 1.0,      # normalized target position
            1.0,                # normalized distance
            1.0,                 # normalized height error
            1.0
        ], dtype=np.float32)
    
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        
        
    def setup_stage_params(self):
        self.params = {
            0: {  # Hovering stage
                'wind': 0.0,
                'area': [5.0, 5.0, 5.0],
                'max_steps': 200,
                'target_type': 'hover',
                'height_range': [-2.0, -1.5],  # Desired hover height
                'target_radius': 0.5
            },
            1: {  # Near vertical movement
                'wind': 0.0,
                'area': [5.0, 5.0, 8.0],
                'max_steps': 250,
                'target_type': 'vertical',
                'height_range': [-4.0, -1.0],
                'target_radius': 0.3
            },
            2: {  # Close-range horizontal movement
                'wind': 0.0,
                'area': [10.0, 10.0, 5.0],
                'max_steps': 300,
                'target_type': 'horizontal_near',
                'target_radius': 0.3,
                'max_target_dist': 5.0
            },
            3: {  # Medium-range movement
                'wind': 0.0,
                'area': [15.0, 15.0, 8.0],
                'max_steps': 400,
                'target_type': 'free_near',
                'target_radius': 0.3,
                'max_target_dist': 10.0
            },
            4: {  # Long-range movement
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 500,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 15.0
            },
            5: {  # Extended range movement
                'wind': 0.0,
                'area': [30.0, 30.0, 15.0],  # Expanded area
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 25.0  # Increased maximum distance
            },
            6: {  # Recovery training
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 500,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 15.0,
                'recovery_interval': 50  # Apply random action every 50 steps
            },
            7: {  # Moderate wind
                'wind': 0.5,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.5,
                'max_target_dist': 25.0
            },
            8: {  # Strong wind
                'wind': 1.0,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.5,  # Slightly larger radius due to strong wind
                'max_target_dist': 25.0
            }
        }
        self.current_params = self.params[self.training_stage]
        
    def _generate_new_target(self):
        """Internal method to generate a new target based on current parameters"""
        params = self.current_params
        
        if params['target_type'] == 'hover':
            return np.array([0.0, 0.0, -1.75])
            
        elif params['target_type'] == 'vertical':
            height = np.random.uniform(*params['height_range'])
            return np.array([0.0, 0.0, height])
            
        elif params['target_type'] == 'horizontal_near':
            angle = np.random.uniform(0, 2*np.pi)
            dist = np.random.uniform(2.0, params['max_target_dist'])
            return np.array([
                dist * np.cos(angle),
                dist * np.sin(angle),
                -2.0
            ])
            
        elif params['target_type'] in ['free_near', 'free']:
            while True:
                point = np.random.uniform(
                    low=[-params['max_target_dist'], -params['max_target_dist'], -4.0],
                    high=[params['max_target_dist'], params['max_target_dist'], -1.0],
                    size=(3,)
                )
                if np.linalg.norm(point - self._get_current_position()) > 2.0:
                    return point

    def _get_target(self):
        """Return the current target"""
        return self.current_target
    
    def _get_distance_target(self):
        return np.linalg.norm(self._get_current_position() - self._get_target())
        
    def _get_current_position(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        pos = kinematics.position

        return np.array([pos.x_val, pos.y_val, pos.z_val], dtype=np.float32)

    def _get_current_velocity(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        vel = kinematics.linear_velocity

        return np.array([vel.x_val, vel.y_val, vel.z_val], dtype=np.float32)
    
    def _get_imu_data(self):
        imu_data = self.client.getImuData()
        # IMU typically gives angular velocity and linear acceleration
        # Store them as [wx, wy, wz, ax, ay, az]
        ang_vel = imu_data.angular_velocity
        lin_acc = imu_data.linear_acceleration
        return np.array([
            ang_vel.x_val, ang_vel.y_val, ang_vel.z_val,
            lin_acc.x_val, lin_acc.y_val, lin_acc.z_val
        ], dtype=np.float32)

    def _get_mag_data(self):
        mag_data = self.client.getMagnetometerData()
        # magnetometer.x, .y, .z
        return np.array([
            mag_data.magnetic_field_body.x_val,
            mag_data.magnetic_field_body.y_val,
            mag_data.magnetic_field_body.z_val
        ], dtype=np.float32)

    def _get_gps_data(self):
        gps_data = self.client.getGpsData()
        # gps_data.gnss.geo_point.latitude, .longitude, .altitude
        return np.array([
            gps_data.gnss.geo_point.latitude,
            gps_data.gnss.geo_point.longitude,
            gps_data.gnss.geo_point.altitude
        ], dtype=np.float32)

    def _get_baro_data(self):
        baro_data = self.client.getBarometerData()
        # baro_data.altitude, baro_data.pressure, baro_data.qnh
        # Just use altitude
        return np.array([baro_data.altitude], dtype=np.float32)

    def _get_observation(self):
        """
        #Get and standardize observations.
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # We'll ignore GPS and magnetometer data as they're less relevant for this task
        # and could add noise to the learning process

        # Standardize position: Scale by area bounds
        normalized_pos = position / np.array(self.current_params['area'])

        # Standardize velocity: Most drones operate within -10 to 10 m/s range
        normalized_vel = np.clip(velocity / 10.0, -1.0, 1.0)

        # Standardize IMU data
        angular_vel = imu_data[:3] / 10.0  # Angular velocity (rad/s)
        linear_acc = imu_data[3:6] / 9.81  # Linear acceleration (normalize by g)

        # Standardize target and distance
        normalized_target = target / np.array(self.current_params['area'])
        normalized_distance = distance / np.linalg.norm(self.current_params['area'])

        # Height error (important for hovering)
        height_error = (position[2] - target[2]) / self.current_params['area'][2]

        # Combine all observations
        observation = np.concatenate([
            normalized_pos,        # [0:3]   - Normalized position (x, y, z)
            normalized_vel,        # [3:6]   - Normalized velocity (vx, vy, vz)
            angular_vel,          # [6:9]   - Normalized angular velocity (wx, wy, wz)
            linear_acc,          # [9:12]  - Normalized linear acceleration (ax, ay, az)
            normalized_target,    # [12:15] - Normalized target position
            [normalized_distance],# [15]    - Normalized distance to target
            [height_error]       # [16]    - Normalized height error
        ])
        # Clip all values to ensure they stay within a reasonable range
        observation = np.clip(observation, -5.0, 5.0)
        
        return observation.astype(np.float32)"""
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # Normalize position relative to the current stage's boundaries
        area_bounds = np.array(self.current_params['area'])
        normalized_pos = np.clip(position / area_bounds, -1.0, 1.0)
        
        # Normalize velocity (typical drone speeds rarely exceed ±10 m/s)
        velocity_scale = np.array([10.0, 10.0, 5.0])  # Less vertical velocity range
        normalized_vel = np.clip(velocity / velocity_scale, -1.0, 1.0)
        
        # Normalize IMU data
        # Angular velocity (typically ±5 rad/s)
        angular_vel = np.clip(imu_data[:3] / 5.0, -1.0, 1.0)
        # Linear acceleration (normalize by g ≈ 9.81 m/s²)
        linear_acc = np.clip(imu_data[3:6] / 9.81, -3.0, 3.0)  # Allow for higher G-forces
        
        # Normalize target position
        normalized_target = np.clip(target / area_bounds, -1.0, 1.0)
        
        # Normalize distance relative to maximum possible distance in current area
        max_possible_distance = np.linalg.norm(area_bounds)
        normalized_distance = np.clip(distance / max_possible_distance, 0.0, 1.0)
        
        # Height error normalization
        max_height_error = area_bounds[2]
        height_error = (position[2] - target[2]) / max_height_error
        normalized_height_error = np.clip(height_error, -1.0, 1.0)

        relative_pos = (target - position) / np.array(self.current_params['area'])
        
        target_direction = (target - position) / (np.linalg.norm(target - position) + 1e-6)
        relative_vel = np.dot(velocity, target_direction)
        
        observation = np.concatenate([
            normalized_pos,        # [0:3]
            normalized_vel,        # [3:6]
            angular_vel,          # [6:9]
            linear_acc,          # [9:12]
            relative_pos,        # [12:15]
            [normalized_distance],# [15]
            [height_error],      # [16]
            [relative_vel]       # [17] - Added velocity towards target
        ])
        return observation.astype(np.float32)

    def _apply_wind(self):
        """Applies a simulated wind force to the drone."""
        wind_x = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_y = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_z = np.random.uniform(-self.wind_factor / 2, self.wind_factor / 2)
        self.client.simSetWind(airsim.Vector3r(wind_x, wind_y, wind_z))
    
    def reset(self, seed=None, options=None):
        if seed is not None:
            self.seed(seed)

        # Reset the drone
        self.client.reset()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        self.client.takeoffAsync().join()

        self.current_step = 0
        self.consecutive_success = 0
        self.episode += 1
        self._apply_wind()
        
        if not hasattr(self, 'current_target') or self.current_target is None:
            self._get_target()  # This will generate a new target
        
        observation = self._get_observation()
        return observation, {}

    def step(self, action):
        self.current_step += 1
        action = np.clip(action, -1.0, 1.0)
        
        if self.training_stage == 6:
            if self.current_step % self.current_params['recovery_interval'] == 0:
                # Apply random action
                random_action = np.random.uniform(-1, 1, size=4)
                self.recovery_action_timer = 10  # Recovery period
                action = random_action
            elif self.recovery_action_timer > 0:
                self.recovery_action_timer -= 1
        # Process controls - separate throttle handling
        max_angles = np.radians([30, 30, 45])  # Reduced pitch/roll angles
        
        pitch = np.clip(action[0] * max_angles[0], -max_angles[0], max_angles[0])
        roll = np.clip(action[1] * max_angles[1], -max_angles[1], max_angles[1])
        
        # Use exponential mapping for throttle to give more control in hover region
        throttle = (action[2])*0.4 + 0.6  # Centered around 0.5
        yaw_rate = np.clip(action[3] * max_angles[2], -max_angles[2], max_angles[2])
        # Add small time delay for stability
        self.client.moveByRollPitchYawrateThrottleAsync(
            roll=roll,
            pitch=pitch,
            yaw_rate=yaw_rate,
            throttle=throttle,
            duration=0.2  # Reduced duration for more frequent updates
        ).join()
        
        self._apply_wind()
        observation = self._get_observation()
        reward, terminated = self._calculate_reward()
        
        truncated = self.current_step >= self.current_params['max_steps']
        #if truncated:
        #    reward += 10
        """
        if self.training_stage == 0:
            # Increment the counter.
            if truncated:
                self.hover_survival_count += 1
                reward += 50

                if self.hover_survival_count >= self.num_consecutive_survivals_needed:
                    self.training_stage = 1
                    self.setup_stage_params()
                    self.hover_survival_count = 0
                    print(
                        f"=== Moved to stage {self.training_stage} after "
                        f"{self.num_consecutive_survivals_needed} consecutive survival episodes. ==="
                    )
        """
        return observation, reward, terminated, truncated, {}
    def _calculate_reward(self):
        pos = self._get_current_position()
        vel = self._get_current_velocity()
        angular_vel = self._get_imu_data()[:3]
        target = self._get_target()
        distance = np.linalg.norm(pos - target)
        reward_scale = 0.1
        reward = 0.0
        # Immediate failure conditions (keep these)
        if self.client.simGetCollisionInfo().has_collided:
            reward -= 100*reward_scale
            return reward, True
        if any(abs(p) > a for p, a in zip(pos, self.current_params['area'])):
            reward -= 50*reward_scale
            return reward, True
        # Main distance reward - smoother gradient
        distance_reward = -distance + 1  # Linear scaling for more consistent gradient
        
        """
        # Stability penalty - much gentler
        stability_penalty = (
            0.05 * np.linalg.norm(angular_vel) +  # Reduced rotation penalty
            0.02 * np.linalg.norm(vel)            # Reduced velocity penalty
        )
        reward -= stability_penalty
        """
        if self.training_stage == 0:  # Hovering
            # Pure hovering - focus on stability
            vertical_distance = abs(pos[2] - target[2])
            reward += (-vertical_distance + 1.0)*reward_scale
            reward += (distance_reward*0.5)*reward_scale
            
            
        elif self.training_stage == 1:  # Vertical movement
            # Reward vertical progress toward target
            vertical_distance = abs(pos[2] - target[2])
            reward += (2.0 * np.exp(-vertical_distance))*reward_scale
            
            # Penalize horizontal drift more strongly
            horizontal_drift = np.linalg.norm(pos[:2] - target[:2])
            reward -= (0.5 * horizontal_drift)*reward_scale
            
            reward += (distance_reward*0.5)*reward_scale
            
        elif self.training_stage == 2:  # Close-range horizontal
            # Reward horizontal progress
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (3.0 * progress)*reward_scale  # Stronger reward for deliberate movement
            self._prev_distance = distance
            
            # Height stability becomes secondary but still important
            reward += (distance_reward)*reward_scale
            
        elif self.training_stage >= 3:  # Medium-range movement
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (2.0 * progress)*reward_scale
            self._prev_distance = distance
            reward += (distance_reward)*reward_scale
        # Success condition
        success_radius = self.current_params['target_radius']
        if distance < success_radius:
            bonus = 0
            bonus += (25.0)*reward_scale
            self.consecutive_success += 1
            
            if self.consecutive_success % self.success_threshold == 0:
                self.current_target = self._generate_new_target()
                bonus += 50.0*reward_scale  
                self.success_count += 1
        
            if self.success_count >= self.stage_threshold:
                self.training_stage = min(8, self.training_stage + 1)
                print(f"\n=== Advanced to stage {self.training_stage} ===")
                self.success_count = 0
                self.setup_stage_params()
                
            return reward, True
        return reward, False

    def close(self):
        self.client.reset()
        self.client.enableApiControl(False)
        self.client.armDisarm(False)

In [4]:
#Train to keep in air

class CosysAirSimEnv_Velocity(gym.Env):
    def __init__(self, training_stage=0):
        super(CosysAirSimEnv_Velocity, self).__init__()

        self.client = MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)

        self.action_space = spaces.Box(low=-1, high=1, shape=(4,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-1.0, high=1.0, shape=(18,), dtype=np.float32)

        self.max_steps = 200
        self.current_step = 0
        self.episode = 0
        self.wind_factor = 0.0
        self.last_throttle = 0.6
        self.success_count = 0
        
        self.success_threshold = 10
        
        self.training_stage = training_stage
        self.setup_stage_params()
        self.stage_progress = 0
        self.stage_threshold = 25
        
        self.hover_survival_count = 0
        self.consecutive_success = 0
        self.current_target = self._generate_new_target()
        
        self.all_time_min_distance = float('inf')
        high = np.array([
            1.0, 1.0, 1.0,      # normalized position
            1.0, 1.0, 1.0,      # normalized velocity
            1.0, 1.0, 1.0,      # normalized angular velocity
            3.0, 3.0, 3.0,      # normalized acceleration (in G's)
            1.0, 1.0, 1.0,      # normalized target position
            1.0,                # normalized distance
            1.0,                 # normalized height error
            1.0
        ], dtype=np.float32)
    
        self.observation_space = spaces.Box(low=-high, high=high, dtype=np.float32)
        
        
    def setup_stage_params(self):
        self.params = {
            0: {  # Hovering stage
                'wind': 0.0,
                'area': [5.0, 5.0, 5.0],
                'max_steps': 100,
                'target_type': 'hover',
                'height_range': [-2.0, -1.5],  # Desired hover height
                'target_radius': 0.5
            },
            1: {  # Near vertical movement
                'wind': 0.0,
                'area': [5.0, 5.0, 8.0],
                'max_steps': 100,
                'target_type': 'vertical',
                'height_range': [-4.0, -1.0],
                'target_radius': 0.3
            },
            2: {  # Close-range horizontal movement
                'wind': 0.0,
                'area': [10.0, 10.0, 5.0],
                'max_steps': 100,
                'target_type': 'horizontal_near',
                'target_radius': 0.3,
                'max_target_dist': 5.0
            },
            3: {  # Medium-range movement
                'wind': 0.0,
                'area': [15.0, 15.0, 8.0],
                'max_steps': 400,
                'target_type': 'free_near',
                'target_radius': 0.3,
                'max_target_dist': 10.0
            },
            4: {  # Long-range movement
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 500,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 15.0
            },
            5: {  # Extended range movement
                'wind': 0.0,
                'area': [30.0, 30.0, 15.0],  # Expanded area
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 25.0  # Increased maximum distance
            },
            6: {  # Recovery training
                'wind': 0.0,
                'area': [20.0, 20.0, 10.0],
                'max_steps': 500,
                'target_type': 'free',
                'target_radius': 0.3,
                'max_target_dist': 15.0,
                'recovery_interval': 50  # Apply random action every 50 steps
            },
            7: {  # Moderate wind
                'wind': 0.5,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.5,
                'max_target_dist': 25.0
            },
            8: {  # Strong wind
                'wind': 1.0,
                'area': [30.0, 30.0, 15.0],
                'max_steps': 600,
                'target_type': 'free',
                'target_radius': 0.5,  # Slightly larger radius due to strong wind
                'max_target_dist': 25.0
            }
        }
        self.current_params = self.params[self.training_stage]
        
    def _generate_new_target(self):
        """Internal method to generate a new target based on current parameters"""
        params = self.current_params
        
        if params['target_type'] == 'hover':
            return np.array([0.0, 0.0, -1.75])
            
        elif params['target_type'] == 'vertical':
            height = np.random.uniform(*params['height_range'])
            return np.array([0.0, 0.0, height])
            
        elif params['target_type'] == 'horizontal_near':
            angle = np.random.uniform(0, 2*np.pi)
            dist = np.random.uniform(2.0, params['max_target_dist'])
            return np.array([
                dist * np.cos(angle),
                dist * np.sin(angle),
                -2.0
            ])
            
        elif params['target_type'] in ['free_near', 'free']:
            while True:
                point = np.random.uniform(
                    low=[-params['max_target_dist'], -params['max_target_dist'], -4.0],
                    high=[params['max_target_dist'], params['max_target_dist'], -1.0],
                    size=(3,)
                )
                if np.linalg.norm(point - self._get_current_position()) > 2.0:
                    return point

    def _get_target(self):
        """Return the current target"""
        return self.current_target
    
    def _get_distance_target(self):
        return np.linalg.norm(self._get_current_position() - self._get_target())
        
    def _get_current_position(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        pos = kinematics.position

        return np.array([pos.x_val, pos.y_val, pos.z_val], dtype=np.float32)

    def _get_current_velocity(self):
        state = self.client.getMultirotorState()
        kinematics = state.kinematics_estimated
        vel = kinematics.linear_velocity

        return np.array([vel.x_val, vel.y_val, vel.z_val], dtype=np.float32)
    
    def _get_imu_data(self):
        imu_data = self.client.getImuData()
        # IMU typically gives angular velocity and linear acceleration
        # We'll store them as [wx, wy, wz, ax, ay, az]
        ang_vel = imu_data.angular_velocity
        lin_acc = imu_data.linear_acceleration
        return np.array([
            ang_vel.x_val, ang_vel.y_val, ang_vel.z_val,
            lin_acc.x_val, lin_acc.y_val, lin_acc.z_val
        ], dtype=np.float32)

    def _get_mag_data(self):
        mag_data = self.client.getMagnetometerData()
        # magnetometer.x, .y, .z
        return np.array([
            mag_data.magnetic_field_body.x_val,
            mag_data.magnetic_field_body.y_val,
            mag_data.magnetic_field_body.z_val
        ], dtype=np.float32)

    def _get_gps_data(self):
        gps_data = self.client.getGpsData()
        # gps_data.gnss.geo_point.latitude, .longitude, .altitude
        return np.array([
            gps_data.gnss.geo_point.latitude,
            gps_data.gnss.geo_point.longitude,
            gps_data.gnss.geo_point.altitude
        ], dtype=np.float32)

    def _get_baro_data(self):
        baro_data = self.client.getBarometerData()
        # baro_data.altitude, baro_data.pressure, baro_data.qnh
        # We'll just use altitude
        return np.array([baro_data.altitude], dtype=np.float32)

    def _get_observation(self):
        """
        #Get and standardize observations.
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # We'll ignore GPS and magnetometer data as they're less relevant for this task
        # and could add noise to the learning process

        # Standardize position: Scale by area bounds
        normalized_pos = position / np.array(self.current_params['area'])

        # Standardize velocity: Most drones operate within -10 to 10 m/s range
        normalized_vel = np.clip(velocity / 10.0, -1.0, 1.0)

        # Standardize IMU data
        angular_vel = imu_data[:3] / 10.0  # Angular velocity (rad/s)
        linear_acc = imu_data[3:6] / 9.81  # Linear acceleration (normalize by g)

        # Standardize target and distance
        normalized_target = target / np.array(self.current_params['area'])
        normalized_distance = distance / np.linalg.norm(self.current_params['area'])

        # Height error (important for hovering)
        height_error = (position[2] - target[2]) / self.current_params['area'][2]

        # Combine all observations
        observation = np.concatenate([
            normalized_pos,        # [0:3]   - Normalized position (x, y, z)
            normalized_vel,        # [3:6]   - Normalized velocity (vx, vy, vz)
            angular_vel,          # [6:9]   - Normalized angular velocity (wx, wy, wz)
            linear_acc,          # [9:12]  - Normalized linear acceleration (ax, ay, az)
            normalized_target,    # [12:15] - Normalized target position
            [normalized_distance],# [15]    - Normalized distance to target
            [height_error]       # [16]    - Normalized height error
        ])
        # Clip all values to ensure they stay within a reasonable range
        observation = np.clip(observation, -5.0, 5.0)
        
        return observation.astype(np.float32)"""
        position = self._get_current_position()
        velocity = self._get_current_velocity()
        imu_data = self._get_imu_data()
        target = self._get_target()
        distance = self._get_distance_target()

        # Normalize position relative to the current stage's boundaries
        area_bounds = np.array(self.current_params['area'])
        normalized_pos = np.clip(position / area_bounds, -1.0, 1.0)
        
        # Normalize velocity (typical drone speeds rarely exceed ±10 m/s)
        velocity_scale = np.array([10.0, 10.0, 5.0])  # Less vertical velocity range
        normalized_vel = np.clip(velocity / velocity_scale, -1.0, 1.0)
        
        # Normalize IMU data
        # Angular velocity (typically ±5 rad/s)
        angular_vel = np.clip(imu_data[:3] / 5.0, -1.0, 1.0)
        # Linear acceleration (normalize by g ≈ 9.81 m/s²)
        linear_acc = np.clip(imu_data[3:6] / 9.81, -3.0, 3.0)  # Allow for higher G-forces
        
        # Normalize target position
        normalized_target = np.clip(target / area_bounds, -1.0, 1.0)
        
        # Normalize distance relative to maximum possible distance in current area
        max_possible_distance = np.linalg.norm(area_bounds)
        normalized_distance = np.clip(distance / max_possible_distance, 0.0, 1.0)
        
        # Height error normalization
        max_height_error = area_bounds[2]
        height_error = (position[2] - target[2]) / max_height_error
        normalized_height_error = np.clip(height_error, -1.0, 1.0)

        relative_pos = (target - position) / np.array(self.current_params['area'])
        
        target_direction = (target - position) / (np.linalg.norm(target - position) + 1e-6)
        relative_vel = np.dot(velocity, target_direction)
        
        observation = np.concatenate([
            normalized_pos,        # [0:3]
            normalized_vel,        # [3:6]
            angular_vel,          # [6:9]
            linear_acc,          # [9:12]
            relative_pos,        # [12:15]
            [normalized_distance],# [15]
            [height_error],      # [16]
            [relative_vel]       # [17] - Added velocity towards target
        ])
        return observation.astype(np.float32)

    def _apply_wind(self):
        """Applies a simulated wind force to the drone."""
        wind_x = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_y = np.random.uniform(-self.wind_factor, self.wind_factor)
        wind_z = np.random.uniform(-self.wind_factor / 2, self.wind_factor / 2)
        self.client.simSetWind(airsim.Vector3r(wind_x, wind_y, wind_z))
    
    def reset(self, seed=None, options=None):
        if seed is not None:
            self.seed(seed)

        # Reset the drone
        self.client.reset()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        self.client.takeoffAsync().join()

        self.current_step = 0
        self.consecutive_success = 0
        self.episode += 1
        self._apply_wind()
        
        if not hasattr(self, 'current_target') or self.current_target is None:
            self._get_target()  # This will generate a new target
        
        observation = self._get_observation()
        return observation, {}

    def step(self, action):
        self.current_step += 1
        action = np.clip(action, -1.0, 1.0)
        
        if self.training_stage == 6:
            if self.current_step % self.current_params['recovery_interval'] == 0:
                # Apply random action
                random_action = np.random.uniform(-1, 1, size=4)
                self.recovery_action_timer = 10  # Recovery period
                action = random_action
            elif self.recovery_action_timer > 0:
                self.recovery_action_timer -= 1
        # Process controls - separate throttle handling
        max_vx, max_vy, max_vz = 3.0, 3.0, 3.0   # [m/s]
        max_yaw_deg = 45.0                      # [deg/s], or you can do rad/s

        vx = float(action[0]) * max_vx
        vy = float(action[1]) * max_vy
        vz = float(action[2]) * max_vz
        yaw_rate_deg = float(action[3]) * max_yaw_deg
        
        yaw_mode = airsim.YawMode(is_rate=True, yaw_or_rate=yaw_rate_deg)

        # Move by velocity in BODY frame
        self.client.moveByVelocityBodyFrameAsync(
            vx=vx,
            vy=vy,
            vz=vz,
            duration=0.25,  # small step
            drivetrain=airsim.DrivetrainType.MaxDegreeOfFreedom,
            yaw_mode=yaw_mode
        ).join()
        
        self._apply_wind()
        observation = self._get_observation()
        reward, terminated = self._calculate_reward()
        
        truncated = self.current_step >= self.current_params['max_steps']
        #if truncated:
        #    reward += 10
        """
        if self.training_stage == 0:
            # If we ended the episode by hitting max_steps (truncate) WITHOUT crashing (terminated=False),
            # then we survived. Increment the counter.
            if truncated:
                self.hover_survival_count += 1
                reward += 50

                # If we've survived enough times in a row, move to stage 1
                if self.hover_survival_count >= self.num_consecutive_survivals_needed:
                    self.training_stage = 1
                    self.setup_stage_params()
                    self.hover_survival_count = 0
                    print(
                        f"=== Moved to stage {self.training_stage} after "
                        f"{self.num_consecutive_survivals_needed} consecutive survival episodes. ==="
                    )
        """
        return observation, reward, terminated, truncated, {}
    def _calculate_reward(self):
        pos = self._get_current_position()
        vel = self._get_current_velocity()
        angular_vel = self._get_imu_data()[:3]
        target = self._get_target()
        distance = np.linalg.norm(pos - target)
        reward_scale = 0.1
        reward = 0.0
        # Immediate failure conditions (keep these)
        if self.client.simGetCollisionInfo().has_collided:
            reward -= 100*reward_scale
            return reward, True
        if any(abs(p) > a for p, a in zip(pos, self.current_params['area'])):
            reward -= 50*reward_scale
            return reward, True
        # Main distance reward - smoother gradient
        distance_reward = -distance+1  # Linear scaling for more consistent gradient
        
        """
        # Stability penalty - much gentler
        stability_penalty = (
            0.05 * np.linalg.norm(angular_vel) +  # Reduced rotation penalty
            0.02 * np.linalg.norm(vel)            # Reduced velocity penalty
        )
        reward -= stability_penalty
        """
        if self.training_stage == 0:  # Hovering
            # Pure hovering - focus on stability
            vertical_distance = abs(pos[2] - target[2])
            reward += (-vertical_distance + 1.0)*reward_scale
            reward += (distance_reward*0.5)*reward_scale
            
            
        elif self.training_stage == 1:  # Vertical movement
            # Reward vertical progress toward target
            vertical_distance = abs(pos[2] - target[2])
            reward += (2.0 * np.exp(-vertical_distance))*reward_scale
            
            # Penalize horizontal drift more strongly
            horizontal_drift = np.linalg.norm(pos[:2] - target[:2])
            reward -= (0.5 * horizontal_drift)*reward_scale
            
            reward += (distance_reward*0.5)*reward_scale
            
        elif self.training_stage == 2:  # Close-range horizontal
            # Reward horizontal progress
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (3.0 * progress)*reward_scale  # Stronger reward for deliberate movement
            self._prev_distance = distance
            
            # Height stability becomes secondary but still important
            reward += (distance_reward)*reward_scale
            
        elif self.training_stage >= 3:  # Medium-range movement
            # Balance distance and control
            if hasattr(self, '_prev_distance'):
                progress = self._prev_distance - distance
                reward += (2.0 * progress)*reward_scale
            self._prev_distance = distance
            reward += (distance_reward)*reward_scale
        # Success condition
        success_radius = self.current_params['target_radius']
        if distance < success_radius:
            bonus = 0
            bonus += (25.0)*reward_scale
            self.consecutive_success += 1
            
            if self.consecutive_success % self.success_threshold == 0:
                self.current_target = self._generate_new_target()
                bonus += 50.0*reward_scale  
                self.success_count += 1
        
            if self.success_count >= self.stage_threshold:
                self.training_stage = min(8, self.training_stage + 1)
                print(f"\n=== Advanced to stage {self.training_stage} ===")
                self.success_count = 0
                self.setup_stage_params()
                
            return reward, True
        return reward, False

    def close(self):
        self.client.reset()
        self.client.enableApiControl(False)
        self.client.armDisarm(False)

In [16]:
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.logger import configure

def train_sb3():
    # 1) Create the environment
    env = CosysAirSimEnv_Basic(training_stage=0)
    
    # 2) Wrap in a VecEnv (SB3 requires vectorized envs)
    vec_env = DummyVecEnv([lambda: env])
    
    log_dir = "tb_logs"
    os.makedirs(log_dir, exist_ok=True)
    
    # 3) Instantiate the RL model (PPO, SAC, TD3, etc.)
    # For continuous actions, PPO or SAC are popular choices
    model = TD3(
        policy="MlpPolicy", 
        env=vec_env, 
        learning_rate=0.01, 
        buffer_size=1000000, 
        learning_starts=100, 
        batch_size=256, 
        tau=0.005, 
        gamma=0.99, 
        train_freq=1, 
        gradient_steps=1, 
        action_noise=None, 
        replay_buffer_class=None, 
        replay_buffer_kwargs=None, 
        optimize_memory_usage=False, 
        policy_delay=2, 
        target_policy_noise=0.2, 
        target_noise_clip=0.5, 
        stats_window_size=100, 
        tensorboard_log=log_dir, 
        policy_kwargs=None, 
        verbose=0, 
        seed=None, 
        device='auto', 
        _init_setup_model=True
    )
    # 4) Train
    model.learn(total_timesteps=200_000, tb_log_name="TD3_airsim")  # or more
    
    # 5) Save
    model.save("TD3_airsim_model")
    print("Training complete! Model saved.")
    
    # 6) Evaluate or Test
    obs, _ = env.reset()
    for _ in range(1000):
        action, _states = model.predict(obs)
        obs, reward, done, truncated, info = env.step(action)
        if done or truncated:
            obs, _ = env.reset()

KeyboardInterrupt: 

In [ ]:
train_sb3()

In [5]:
class ReplayBuffer:
    def __init__(self, max_size, obs_dim, act_dim, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0
        self.device = device
        # Use float32 for all arrays to match network precision
        self.states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.actions = np.zeros((max_size, act_dim), dtype=np.float32)
        self.rewards = np.zeros(max_size, dtype=np.float32)  # Flattened array
        self.next_states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.dones = np.zeros(max_size, dtype=np.float32)    # Flattened array

    def add(self, state, action, reward, next_state, done):
        # Ensure incoming data is on CPU and in numpy format
        if torch.is_tensor(state):
            state = state.detach().cpu().numpy()
        if torch.is_tensor(action):
            action = action.detach().cpu().numpy()
        if torch.is_tensor(reward):
            reward = reward.detach().cpu().numpy()
        if torch.is_tensor(next_state):
            next_state = next_state.detach().cpu().numpy()
        if torch.is_tensor(done):
            done = done.detach().cpu().numpy()
            
        np.copyto(self.states[self.ptr], state)
        np.copyto(self.actions[self.ptr], action)
        self.rewards[self.ptr] = reward
        np.copyto(self.next_states[self.ptr], next_state)
        self.dones[self.ptr] = done
        
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)
    
    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)
        
        return (
            torch.FloatTensor(self.states[ind]).to(self.device),
            torch.FloatTensor(self.actions[ind]).to(self.device),
            torch.FloatTensor(self.rewards[ind]).to(self.device),
            torch.FloatTensor(self.next_states[ind]).to(self.device),
            torch.FloatTensor(self.dones[ind]).to(self.device)
        )
    
    def save(self, path):
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the buffer state and metadata
        save_dict = {
            'max_size': self.max_size,
            'ptr': self.ptr,
            'size': self.size,
            'states': self.states,
            'actions': self.actions,
            'rewards': self.rewards,
            'next_states': self.next_states,
            'dones': self.dones,
            'device': self.device
        }
        
        try:
            with open(path, 'wb') as f:
                pickle.dump(save_dict, f)
        except Exception as e:
            raise RuntimeError(f"Failed to save buffer to {path}: {e}")

    @staticmethod
    def load(path, device=None):
        try:
            with open(path, 'rb') as f:
                save_dict = pickle.load(f)
            
            # Create new buffer with saved dimensions
            obs_dim = save_dict['states'].shape[1]
            act_dim = save_dict['actions'].shape[1]
            buffer = ReplayBuffer(
                max_size=save_dict['max_size'],
                obs_dim=obs_dim,
                act_dim=act_dim,
                device=device or save_dict['device']
            )
            
            # Restore buffer state
            buffer.ptr = save_dict['ptr']
            buffer.size = save_dict['size']
            buffer.states = save_dict['states']
            buffer.actions = save_dict['actions']
            buffer.rewards = save_dict['rewards']
            buffer.next_states = save_dict['next_states']
            buffer.dones = save_dict['dones']
            
            return buffer
        except Exception as e:
            raise RuntimeError(f"Failed to load buffer from {path}: {e}")
        
    def __len__(self):
        return self.size
"""
class ReplayBuffer:
    def __init__(self, max_size, obs_dim, act_dim, device="cuda" if torch.cuda.is_available() else "cpu"):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0
        self.device = device

        # Initialize buffers with correct shapes
        self.states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.actions = np.zeros((max_size, act_dim), dtype=np.float32)
        self.rewards = np.zeros((max_size, 1), dtype=np.float32)  # Changed shape to (max_size, 1)
        self.next_states = np.zeros((max_size, obs_dim), dtype=np.float32)
        self.dones = np.zeros((max_size, 1), dtype=np.float32)    # Changed shape to (max_size, 1)

    def add(self, state, action, reward, next_state, done):
        # Convert inputs to numpy arrays and ensure correct shapes
        state = np.array(state, dtype=np.float32).flatten()
        action = np.array(action, dtype=np.float32).flatten()
        reward = np.array(reward, dtype=np.float32).reshape(1)
        next_state = np.array(next_state, dtype=np.float32).flatten()
        done = np.array(done, dtype=np.float32).reshape(1)

        # Store transition
        self.states[self.ptr] = state
        self.actions[self.ptr] = action
        self.rewards[self.ptr] = reward
        self.next_states[self.ptr] = next_state
        self.dones[self.ptr] = done

        # Update pointer and size
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = np.random.randint(0, self.size, size=batch_size)
        
        return (
            torch.FloatTensor(self.states[ind]).to(self.device),
            torch.FloatTensor(self.actions[ind]).to(self.device),
            torch.FloatTensor(self.rewards[ind]).to(self.device),
            torch.FloatTensor(self.next_states[ind]).to(self.device),
            torch.FloatTensor(self.dones[ind]).to(self.device)
        )
    def save(self, path):
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the buffer state and metadata
        save_dict = {
            'max_size': self.max_size,
            'ptr': self.ptr,
            'size': self.size,
            'states': self.states,
            'actions': self.actions,
            'rewards': self.rewards,
            'next_states': self.next_states,
            'dones': self.dones,
            'device': self.device
        }
        
        try:
            with open(path, 'wb') as f:
                pickle.dump(save_dict, f)
        except Exception as e:
            raise RuntimeError(f"Failed to save buffer to {path}: {e}")

    @staticmethod
    def load(path, device=None):
        try:
            with open(path, 'rb') as f:
                save_dict = pickle.load(f)
            
            # Create new buffer with saved dimensions
            obs_dim = save_dict['states'].shape[1]
            act_dim = save_dict['actions'].shape[1]
            buffer = ReplayBuffer(
                max_size=save_dict['max_size'],
                obs_dim=obs_dim,
                act_dim=act_dim,
                device=device or save_dict['device']
            )
            
            # Restore buffer state
            buffer.ptr = save_dict['ptr']
            buffer.size = save_dict['size']
            buffer.states = save_dict['states']
            buffer.actions = save_dict['actions']
            buffer.rewards = save_dict['rewards']
            buffer.next_states = save_dict['next_states']
            buffer.dones = save_dict['dones']
            
            return buffer
        except Exception as e:
            raise RuntimeError(f"Failed to load buffer from {path}: {e}")
        
    def __len__(self):
        return self.size
"""
class TD3Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, max_action, hidden_dim=8192):
        super().__init__()
        self.feature_net = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU()
        )
        
        self.action_net = nn.Sequential(
            nn.Linear(hidden_dim, act_dim),
            nn.Tanh()
        )
        
        self.max_action = max_action
        self.__init_args__ = (obs_dim, act_dim, max_action, hidden_dim)
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                nn.init.constant_(m.bias, 0)

    def forward(self, state):
        features = self.feature_net(state)
        return self.max_action * self.action_net(features)

class TD3Critic(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_dim=4096):
        super().__init__()
        self.state_net = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
        )
        
        self.action_net = nn.Sequential(
            nn.Linear(act_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
        )
        
        self.q_net = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
        self.__init_args__ = (obs_dim, act_dim, hidden_dim)
        self._init_weights()
        
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.orthogonal_(m.weight, gain=1.0)
                nn.init.constant_(m.bias, 0)

    def forward(self, state, action):
        state_features = self.state_net(state)
        action_features = self.action_net(action)
        features = torch.cat([state_features, action_features], dim=-1)
        return self.q_net(features)
    """
class TD3Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, max_action, hidden_dim=256):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(obs_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, act_dim),
            nn.Tanh()
        )
        
        self.max_action = max_action
        self.__init_args__ = (obs_dim, act_dim, max_action, hidden_dim)
        
        # Use orthogonal initialization
        for layer in self.net:
            if isinstance(layer, nn.Linear):
                nn.init.orthogonal_(layer.weight, gain=np.sqrt(2))
                nn.init.constant_(layer.bias, 0.0)

    def forward(self, state):
        return self.max_action * self.net(state)

class TD3Critic(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_dim=256):
        super().__init__()
        
        # Q1 architecture
        self.q1_net = nn.Sequential(
            nn.Linear(obs_dim + act_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
        # Q2 architecture
        self.q2_net = nn.Sequential(
            nn.Linear(obs_dim + act_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
        
        self.__init_args__ = (obs_dim, act_dim, hidden_dim)
        self._init_weights()

    def _init_weights(self):
        for net in [self.q1_net, self.q2_net]:
            for m in net.modules():
                if isinstance(m, nn.Linear):
                    nn.init.orthogonal_(m.weight, gain=np.sqrt(2))
                    nn.init.constant_(m.bias, 0)

    def forward(self, state, action):
        # Ensure inputs are properly shaped
        if state.dim() == 1:
            state = state.unsqueeze(0)
        if action.dim() == 1:
            action = action.unsqueeze(0)
            
        # Concatenate state and action
        sa = torch.cat([state, action], dim=1)
        
        # Get Q-values
        q1 = self.q1_net(sa)
        q2 = self.q2_net(sa)
        
        return q1, q2

    def Q1(self, state, action):
        # Ensure inputs are properly shaped
        if state.dim() == 1:
            state = state.unsqueeze(0)
        if action.dim() == 1:
            action = action.unsqueeze(0)
            
        # Concatenate state and action
        sa = torch.cat([state, action], dim=1)
        
        return self.q1_net(sa)
"""
class TD3Trainer:
    def __init__(self, actor, critic1, critic2, actor_optimizer, critic_optimizer1, 
                 critic_optimizer2, max_action, device, gamma=0.99, tau=0.001):
        
        self.actor = actor
        self.critic1 = critic1
        self.critic2 = critic2
        self.actor_target = type(actor)(*actor.__init_args__).to(device)
        self.critic_target1 = type(critic1)(*critic1.__init_args__).to(device)
        self.critic_target2 = type(critic2)(*critic2.__init_args__).to(device)
        
        self.actor_optimizer = actor_optimizer
        self.critic_optimizer1 = critic_optimizer1
        self.critic_optimizer2 = critic_optimizer2
        
        self.max_action = max_action
        self.device = device
        self.gamma = gamma
        self.tau = tau
        
        self.target_entropy = -float(actor.__init_args__[1])
        
        # Initialize episode tracking
        self.current_episode_rewards = []
        self.episode_returns = []
        
    def _hard_update_targets(self):
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.critic_target1.load_state_dict(self.critic1.state_dict())
        self.critic_target2.load_state_dict(self.critic2.state_dict())
    
    def _soft_update(self, target, source):
        with torch.no_grad():
            for target_param, param in zip(target.parameters(), source.parameters()):
                target_param.data.copy_(
                    self.tau * param.data + (1.0 - self.tau) * target_param.data
                )

    def train_step(self, replay_buffer, batch_size, noise_std, noise_clip, policy_delay, total_it, noise):
        state, action, reward, next_state, done = replay_buffer.sample(batch_size)
        reward = reward.view(-1)
        done = done.view(-1)

        with torch.no_grad():
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)
            
            target_Q1 = self.critic_target1(next_state, next_action)
            target_Q2 = self.critic_target2(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            
            if len(reward.shape) == 1:
                reward = reward.unsqueeze(-1)
            if len(done.shape) == 1:
                done = done.unsqueeze(-1)
            target_Q = reward + (1 - done) * self.gamma * target_Q
        
        # Critic 1 update
        current_Q1 = self.critic1(state, action)
        critic_mse_loss1 = F.mse_loss(current_Q1, target_Q)
        critic_l2_reg1 = 0.00001 * sum(torch.sum(param ** 2) for param in self.critic1.parameters())
        critic_loss1 = critic_mse_loss1 + critic_l2_reg1
        
        self.critic_optimizer1.zero_grad()
        critic_loss1.backward(retain_graph=True) 
        torch.nn.utils.clip_grad_norm_(self.critic1.parameters(), 1.0)
        self.critic_optimizer1.step()
        
        # Critic 2 update
        current_Q2 = self.critic2(state, action)
        critic_mse_loss2 = F.mse_loss(current_Q2, target_Q)
        critic_l2_reg2 = 0 #0.00001 * sum(torch.sum(param ** 2) for param in self.critic2.parameters())
        critic_loss2 = critic_mse_loss2 + critic_l2_reg2
        
        self.critic_optimizer2.zero_grad()
        critic_loss2.backward()  
        torch.nn.utils.clip_grad_norm_(self.critic2.parameters(), 1.0)
        self.critic_optimizer2.step()

        actor_loss = None
        if total_it % policy_delay == 0:
            actor_actions = self.actor(state)
            actor_loss = -self.critic1(state, actor_actions).mean()
            
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 1.0)
            self.actor_optimizer.step()
            
            self._soft_update(self.actor_target, self.actor)
            self._soft_update(self.critic_target1, self.critic1)
            self._soft_update(self.critic_target2, self.critic2)
        
        return {
            'critic_loss1': critic_loss1.item(),
            'critic_loss2': critic_loss2.item(),
            'actor_loss': actor_loss.item() if actor_loss is not None else None
        }
    """
class TD3Trainer:
    def __init__(self, actor, critic1, critic2, actor_optimizer, critic_optimizer1, 
                 critic_optimizer2, max_action, device, gamma=0.99, tau=0.005):
        
        self.actor = actor
        self.critic1 = critic1
        self.critic2 = critic2
        self.actor_target = type(actor)(*actor.__init_args__).to(device)
        self.critic_target1 = type(critic1)(*critic1.__init_args__).to(device)
        self.critic_target2 = type(critic2)(*critic2.__init_args__).to(device)
        
        self.actor_optimizer = actor_optimizer
        self.critic_optimizer1 = critic_optimizer1
        self.critic_optimizer2 = critic_optimizer2
        
        self.max_action = max_action
        self.device = device
        self.gamma = gamma
        self.tau = tau
        
        self._hard_update_targets()

    def _hard_update_targets(self):
        self.actor_target.load_state_dict(self.actor.state_dict())
        self.critic_target1.load_state_dict(self.critic1.state_dict())
        self.critic_target2.load_state_dict(self.critic2.state_dict())
    
    def _soft_update(self, target, source):
        with torch.no_grad():
            for target_param, param in zip(target.parameters(), source.parameters()):
                target_param.data.copy_(
                    self.tau * param.data + (1.0 - self.tau) * target_param.data
                )

    def train_step(self, replay_buffer, batch_size, noise_std, noise_clip, policy_delay, total_it):
        # Sample from replay buffer
        state, action, reward, next_state, done = replay_buffer.sample(batch_size)
        
        # Ensure proper dimensions
        reward = reward.view(-1, 1)
        done = done.view(-1, 1)

        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (torch.randn_like(action) * noise_std).clamp(-noise_clip, noise_clip)
            next_action = (self.actor_target(next_state) + noise).clamp(-self.max_action, self.max_action)
            
            # Compute the target Q value
            target_Q1, target_Q2 = self.critic_target1(next_state, next_action)
            target_Q = torch.min(target_Q1, target_Q2)
            target_Q = reward + (1 - done) * self.gamma * target_Q

        # Get current Q estimates
        current_Q1, _ = self.critic1(state, action)
        current_Q2, _ = self.critic2(state, action)

        # Compute critic loss
        critic_loss1 = F.mse_loss(current_Q1, target_Q)
        critic_loss2 = F.mse_loss(current_Q2, target_Q)

        # Optimize the critics
        self.critic_optimizer1.zero_grad()
        critic_loss1.backward()
        self.critic_optimizer1.step()

        self.critic_optimizer2.zero_grad()
        critic_loss2.backward()
        self.critic_optimizer2.step()

        actor_loss = None

        # Delayed policy updates
        if total_it % policy_delay == 0:
            # Compute actor loss
            actor_action = self.actor(state)
            Q1, _ = self.critic1(state, actor_action)
            actor_loss = -Q1.mean()

            # Optimize the actor
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Update the frozen target models
            self._soft_update(self.actor_target, self.actor)
            self._soft_update(self.critic_target1, self.critic1)
            self._soft_update(self.critic_target2, self.critic2)

        return {
            'critic_loss1': critic_loss1.item(),
            'critic_loss2': critic_loss2.item(),
            'actor_loss': actor_loss.item() if actor_loss is not None else None
        }
    """

In [11]:
def compute_gradient_norm(model):
    total_norm = 0
    for p in model.parameters():
        if p.grad is not None:
            param_norm = p.grad.data.norm(2)
            total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

def compute_parameter_norm(model):
    total_norm = 0
    for p in model.parameters():
        param_norm = p.data.norm(2)
        total_norm += param_norm.item() ** 2
    return total_norm ** 0.5

def evaluate_policy(actor, env, num_episodes=10, device='cuda'):
    """Evaluate the policy without exploration noise"""
    eval_rewards = []
    eval_success = 0
    
    for _ in range(num_episodes):
        state, _ = env.reset()
        episode_reward = 0
        done = False
        
        while not done:
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                action = actor(state_tensor).cpu().numpy().flatten()
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            episode_reward += reward
            state = next_state
            
            if info.get('success', False):
                eval_success += 1
                break
                
        eval_rewards.append(episode_reward)
    
    return {
        'mean_reward': np.mean(eval_rewards),
        'std_reward': np.std(eval_rewards),
        'success_rate': eval_success / num_episodes,
        'rewards': eval_rewards
    }

def run_training(env, obs_dim, act_dim, max_action, episodes=10000):
    # Initialize environment and models
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Initialize TensorBoard writer with more descriptive name
    current_time = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    log_dir = os.path.join('logs', f'TD3_train_{current_time}')
    train_writer = SummaryWriter(log_dir + '/train')
    eval_writer = SummaryWriter(log_dir + '/eval')
    print(f"TensorBoard logs will be saved to: {log_dir}")
    
    # Model initialization
    actor = TD3Actor(obs_dim, act_dim, max_action).to(device)
    critic1 = TD3Critic(obs_dim, act_dim).to(device)
    critic2 = TD3Critic(obs_dim, act_dim).to(device)
    
    actor_optimizer = optim.Adam(actor.parameters(), lr=3e-2)
    critic_optimizer1 = optim.Adam(critic1.parameters(), lr=3e-2)
    critic_optimizer2 = optim.Adam(critic2.parameters(), lr=3e-2)

    # Load checkpoint if exists
    total_steps = 0
    current_stage = 0
    best_rewards = {i: float('-inf') for i in range(9)}
    
    try:
        checkpoint = torch.load('models/TD3/Model/mtp_model.pth')
        actor.load_state_dict(checkpoint['actor_state_dict'])
        critic1.load_state_dict(checkpoint['critic1_state_dict'])
        critic2.load_state_dict(checkpoint['critic2_state_dict'])
        actor_optimizer.load_state_dict(checkpoint['actor_optimizer_state_dict'])
        critic_optimizer1.load_state_dict(checkpoint['critic1_optimizer_state_dict'])
        critic_optimizer2.load_state_dict(checkpoint['critic2_optimizer_state_dict'])
        
        try:
            total_steps = checkpoint['total_steps']
            current_stage = checkpoint['stage']
            best_rewards = checkpoint['best_rewards']
        except KeyError:
            pass
        print(f"Loaded checkpoint successfully. Total steps: {total_steps}")
    except:
        print("No checkpoint found, starting fresh")

    # Initialize or load replay buffer
    try:
        replay_buffer = ReplayBuffer.load('models/TD3/Replay_Buffer/mtp_replay_buffer.pkl')
        print("Loaded buffer successfully")
    except:
        replay_buffer = ReplayBuffer(max_size=1_000_000, obs_dim=obs_dim, act_dim=act_dim)
        print("No buffer found, starting fresh")
    
    trainer = TD3Trainer(
        actor=actor,
        critic1=critic1,
        critic2=critic2,
        actor_optimizer=actor_optimizer,
        critic_optimizer1=critic_optimizer1,
        critic_optimizer2=critic_optimizer2,
        max_action=max_action,
        device=device
    )
    trainer._hard_update_targets()

    # Training hyperparameters
    batch_size = 256
    warmup_steps = 10000
    noise_std = 0.1
    noise_clip = 0.5
    policy_delay = 2
    exploration_noise = 0.2
    eval_freq = 1000  # Evaluate every 1000 episodes
    
    # Create directories
    os.makedirs('models/TD3/Replay_Buffer', exist_ok=True)
    os.makedirs('models/TD3/Model', exist_ok=True)
    os.makedirs('logs', exist_ok=True)
    
    # Training metrics tracking
    episode_rewards = []
    recent_rewards = deque(maxlen=100)
    stage_rewards = {i: [] for i in range(9)}
    best_eval_reward = float('-inf')
    
    def get_exploration_noise(action):
        base_noise = (torch.randn_like(action) * noise_std).clamp(-noise_clip, noise_clip)
        return base_noise
    
    def log_training_step(train_info, step):
        # Log losses
        train_writer.add_scalar('Loss/Critic1', train_info['critic_loss1'], step)
        train_writer.add_scalar('Loss/Critic2', train_info['critic_loss2'], step)
        if train_info['actor_loss'] is not None:
            train_writer.add_scalar('Loss/Actor', train_info['actor_loss'], step)
        
        # Log gradient norms
        train_writer.add_scalar('Gradients/Actor_Norm', compute_gradient_norm(actor), step)
        train_writer.add_scalar('Gradients/Critic1_Norm', compute_gradient_norm(critic1), step)
        train_writer.add_scalar('Gradients/Critic2_Norm', compute_gradient_norm(critic2), step)
        
        # Log parameter norms
        train_writer.add_scalar('Parameters/Actor_Norm', compute_parameter_norm(actor), step)
        train_writer.add_scalar('Parameters/Critic1_Norm', compute_parameter_norm(critic1), step)
        train_writer.add_scalar('Parameters/Critic2_Norm', compute_parameter_norm(critic2), step)
        
        # Log learning rates
        train_writer.add_scalar('LearningRate/Actor', actor_optimizer.param_groups[0]['lr'], step)
        train_writer.add_scalar('LearningRate/Critic1', critic_optimizer1.param_groups[0]['lr'], step)
        train_writer.add_scalar('LearningRate/Critic2', critic_optimizer2.param_groups[0]['lr'], step)
    
    def run_evaluation(episode):
        # Create a separate environment for evaluation
        eval_env = env.__class__()  # Assuming env has a constructor that takes no arguments
        eval_env.training_stage = env.training_stage  # Sync the training stage
        
        # Run evaluation
        eval_results = evaluate_policy(actor, eval_env, num_episodes=10, device=device)
        
        # Log evaluation metrics
        eval_writer.add_scalar('Eval/Mean_Reward', eval_results['mean_reward'], episode)
        eval_writer.add_scalar('Eval/Reward_Std', eval_results['std_reward'], episode)
        eval_writer.add_scalar('Eval/Success_Rate', eval_results['success_rate'], episode)
        
        # Log reward distribution
        eval_writer.add_histogram('Eval/Reward_Distribution', 
                                torch.tensor(eval_results['rewards']), 
                                episode)
        
        eval_env.close()
        return eval_results
    
    progress_bar = tqdm.tqdm(range(episodes), desc="Training")
    for episode in progress_bar:
        state, _ = env.reset()
        episode_reward = 0
        done = False
        episode_steps = 0
        current_stage = env.training_stage
        
        # Log current stage
        train_writer.add_scalar('Training/Current_Stage', current_stage, total_steps)
        
        while not done:
            # Select action
            if total_steps < warmup_steps:
                action = np.random.uniform(-max_action, max_action, size=act_dim)
                train_writer.add_scalar('Training/Exploration_Type', 0, total_steps)  # 0 for random
            else:
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
                    action = actor(state_tensor).cpu().numpy().flatten()
                    current_noise = get_exploration_noise(action)
                    action += np.random.normal(0, current_noise, size=act_dim)
                action = np.clip(action, -max_action, max_action)
                train_writer.add_scalar('Training/Exploration_Type', 1, total_steps)  # 1 for policy
                train_writer.add_scalar('Training/Exploration_Noise', current_noise, total_steps)
            
            # Log action statistics
            train_writer.add_histogram('Actions/Distribution', action, total_steps)
            train_writer.add_scalar('Actions/Mean', np.mean(action), total_steps)
            train_writer.add_scalar('Actions/Std', np.std(action), total_steps)
            
            # Step environment
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            
            # Store transition
            replay_buffer.add(state, action, reward, next_state, float(done))
            
            # Update state and metrics
            state = next_state
            episode_reward += reward
            total_steps += 1
            episode_steps += 1
            
            # Train agent
            if total_steps > warmup_steps and len(replay_buffer.states) > batch_size:
                train_info = trainer.train_step(
                    replay_buffer=replay_buffer,
                    batch_size=batch_size,
                    noise_std=noise_std,
                    noise_clip=noise_clip,
                    policy_delay=policy_delay,
                    total_it=total_steps,
                    action = action,
                    reward = reward,
                    noise = current_noise
                )
                
                # Log training metrics
                log_training_step(train_info, total_steps)
        
        # Episode completion logging
        recent_rewards.append(episode_reward)
        episode_rewards.append(episode_reward)
        stage_rewards[current_stage].append(episode_reward)
        
        # Log episode metrics
        train_writer.add_scalar('Episode/Reward', episode_reward, episode)
        train_writer.add_scalar('Episode/Steps', episode_steps, episode)
        train_writer.add_scalar('Episode/Average_100_Episodes', np.mean(recent_rewards), episode)
        train_writer.add_scalar('Training/Success_Count', env.success_count, episode)
        train_writer.add_scalar('Training/Buffer_Size', len(replay_buffer), episode)
        
        # Run evaluation periodically
        if episode % eval_freq == 0:
            eval_results = run_evaluation(episode)
            
            # Save best model based on evaluation
            if eval_results['mean_reward'] > best_eval_reward:
                best_eval_reward = eval_results['mean_reward']
                torch.save({
                    'actor_state_dict': actor.state_dict(),
                    'critic1_state_dict': critic1.state_dict(),
                    'critic2_state_dict': critic2.state_dict(),
                    'actor_optimizer_state_dict': actor_optimizer.state_dict(),
                    'critic1_optimizer_state_dict': critic_optimizer1.state_dict(),
                    'critic2_optimizer_state_dict': critic_optimizer2.state_dict(),
                    'total_steps': total_steps,
                    'episode': episode,
                    'stage': current_stage,
                    'eval_reward': best_eval_reward,
                    'best_rewards': best_rewards
                }, 'models/TD3/Model/mtp_model_best_eval.pth')
        
        # Stage-specific metrics
        if len(stage_rewards[current_stage]) >= 100:
            current_avg_reward = np.mean(stage_rewards[current_stage][-100:])
            train_writer.add_scalar(f'Stage_{current_stage}/Average_100_Episodes', 
                                  current_avg_reward, 
                                  len(stage_rewards[current_stage]))
            
            if current_avg_reward > best_rewards[current_stage]:
                best_rewards[current_stage] = current_avg_reward
                train_writer.add_scalar(f'Stage_{current_stage}/Best_Average_Reward', 
                                      current_avg_reward, 
                                      len(stage_rewards[current_stage]))
        
        # Update progress bar
        progress_bar.set_postfix({
            'stage': current_stage,
            'reward': f'{episode_reward:.2f}',
            'avg_reward': f'{np.mean(recent_rewards):.2f}' if episode_rewards else 'N/A',
            'success_count': env.success_count
        })
        
        # Check for training completion
        if current_stage == 8 and env.success_count >= env.stage_threshold:
            print("\n=== Training completed successfully! ===")
            env.close()
            print("Environment closed. Running final evaluation...")
            final_eval_results = run_evaluation(episode)
            print(f"Final evaluation results: {final_eval_results}")
            train_writer.close()
            eval_writer.close()
            break
        
        # Periodic checkpointing
        if episode % 2500 == 0:
            checkpoint_path = f'models/TD3/Model/mtp_backup_{episode}.pth'
            torch.save({
                'actor_state_dict': actor.state_dict(),
                'critic1_state_dict': critic1.state_dict(),
                'critic2_state_dict': critic2.state_dict(),
                'actor_optimizer_state_dict': actor_optimizer.state_dict(),
                'critic1_optimizer_state_dict': critic_optimizer1.state_dict(),
                'critic2_optimizer_state_dict': critic_optimizer2.state_dict(),
                'episode': episode,
                'total_steps': total_steps,
                }, checkpoint_path)
            print(f"Checkpoint saved at {checkpoint_path}")
    return actor, critic1, critic2, episode_rewards

In [12]:
env = CosysAirSimEnv_Velocity(training_stage=0)

# Get environment dimensions
obs_dim = env.observation_space.shape[0]  
act_dim = env.action_space.shape[0]     
max_action = float(env.action_space.high[0]) 

# Run training
actor, critic1, critic2, rewards = run_training(
    env=env,
    obs_dim=obs_dim,
    act_dim=act_dim,
    max_action=max_action,
    episodes= 10_000
)

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)

Using device: cuda
TensorBoard logs will be saved to: logs\TD3_train_20250117-221922


C:\Users\henhe\AppData\Local\Temp\ipykernel_64508\2133005428.py:75: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('models/TD3/Model/mtp_model.pth')


No checkpoint found, starting fresh
No buffer found, starting fresh


Training:   0%|          | 0/10000 [00:00<?, ?it/s]

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)



...

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)



Training:  90%|█████████ | 9000/10000 [7:42:51<1:06:58,  4.02s/it, stage=0, reward=-10.00, avg_reward=-9.98, success_count=0] 

Connected!
Client Ver:3 (Min Req: 3), Server Ver:3 (Min Req: 3)



Training: 100%|██████████| 10000/10000 [8:32:50<00:00,  3.08s/it, stage=0, reward=-10.00, avg_reward=-9.90, success_count=0]  


TypeError: cannot unpack non-iterable NoneType object